In [1]:
import random
import copy

# ----------------------------------------
# WEATHER STATES
# ----------------------------------------

weather_states = ["rain", "snow", "clear", "humid"]

# ----------------------------------------
# TEMPERATURE SELECTION
# ----------------------------------------

def select_temperature(weather):
    if weather == "rain":
        return random.choice(["cool", "mild"])
    elif weather == "snow":
        return "cold"
    elif weather == "clear":
        return random.choice(["mild", "warm", "hot"])
    elif weather == "humid":
        return random.choice(["humid", "warm"])

# ----------------------------------------
# MODEL-BASED REFLEX AGENT
# ----------------------------------------

class SprinklerAgent:

    def __init__(self):
        self.state = {
            "weather": None,
            "temperature": None,
            "soil_moisture": "dry",
            "pressure": None,
            "cycle": 0
        }

    # ----------------------------------------
    # UPDATE INTERNAL STATE FROM PERCEPT
    # ----------------------------------------
    def update_state(self, weather, temperature):

        self.state["weather"] = weather
        self.state["temperature"] = temperature

        # Soil initialization
        if weather in ["rain", "snow"]:
            self.state["soil_moisture"] = "wet"
        else:
            self.state["soil_moisture"] = "dry"

    # ----------------------------------------
    # PRESSURE MODEL
    # ----------------------------------------
    def update_pressure(self):

        weather = self.state["weather"]
        temperature = self.state["temperature"]
        cycle = self.state["cycle"]

        if weather in ["rain", "snow"]:
            self.state["pressure"] = "off"

        elif temperature == "hot":
            self.state["pressure"] = "high"

        elif weather == "humid":
            if cycle == 0:
                self.state["pressure"] = "high"
            else:
                self.state["pressure"] = "low"

        elif temperature == "warm":
            self.state["pressure"] = "high"

        elif temperature == "mild":
            self.state["pressure"] = "low"

        else:
            self.state["pressure"] = "low"

    # ----------------------------------------
    # REFLEX ACTION SELECTION
    # ----------------------------------------
    def choose_action(self):

        weather = self.state["weather"]
        temperature = self.state["temperature"]
        pressure = self.state["pressure"]
        cycle = self.state["cycle"]

        if weather in ["rain", "snow"]:
            return "off", 0

        if temperature == "hot":
            return "on", 20

        if weather == "humid":
            return ("on", 20) if cycle == 0 else ("on", 10)

        if temperature == "warm":
            return ("on", 20) if cycle == 0 else ("on", 10)

        if temperature == "mild":
            return ("on", 20) if cycle == 0 else ("off", 0)

        return "off", 0

    # ----------------------------------------
    # UPDATE INTERNAL STATE AFTER ACTION
    # ----------------------------------------
    def update_after_action(self, action):

        if action == "on":
            self.state["soil_moisture"] = "wet"

        self.state["cycle"] += 1


# ----------------------------------------
# SIMULATION
# ----------------------------------------

def run_simulation():

    print("\nEnter weather for 5 days (rain/snow/clear/humid)\n")

    daily_inputs = []
    for i in range(5):
        w = input(f"Day {i+1} Weather: ").lower()
        while w not in weather_states:
            print("Invalid input. Choose from rain/snow/clear/humid.")
            w = input(f"Day {i+1} Weather: ").lower()
        daily_inputs.append(w)

    print("\n========== MODEL-BASED REFLEX AGENT OUTPUT ==========\n")

    for day in range(5):

        print(f"================ DAY {day+1} ================\n")

        agent = SprinklerAgent()

        weather = daily_inputs[day]
        temperature = select_temperature(weather)

        agent.update_state(weather, temperature)

        print("Initial Internal State:")
        print(agent.state)
        print()

        cycles = 1 if weather in ["rain", "snow"] else 2

        for c in range(cycles):

            print(f"----- Cycle {c+1} -----")

            # Update pressure
            agent.update_pressure()

            print("Internal State BEFORE Action:")
            print(agent.state)

            action, minutes = agent.choose_action()

            print("\nChosen Action:")
            print(f"Sprinkler: {action}")
            print(f"Irrigation Time: {minutes} minutes")

            # Simulate irrigation
            if action == "on":
                print("\nSprinkler running...")
                print(f"Watering for {minutes} minutes...")

                # Internal state changes
                agent.update_after_action(action)

                print("\nInternal State AFTER Irrigation:")
                print(agent.state)

                print("\nSince soil is now wet, sprinkler turns OFF automatically.")

            else:
                agent.update_after_action(action)
                print("\nNo irrigation required.")

            print("\n-----------------------------------------\n")

            # Skip second cycle for mild day
            if temperature == "mild" and c == 0:
                print("Cycle 2 skipped (mild day sufficient irrigation)\n")
                break

        print("\n\n")


# ----------------------------------------
# RUN
# ----------------------------------------

run_simulation()


Enter weather for 5 days (rain/snow/clear/humid)



Day 1 Weather:  rain
Day 2 Weather:  clear
Day 3 Weather:  clear
Day 4 Weather:  humid
Day 5 Weather:  humid



========== MODEL-BASED REFLEX AGENT OUTPUT ==========

================ DAY 1 ================

Initial Internal State:
{'weather': 'rain', 'temperature': 'mild', 'soil_moisture': 'wet', 'pressure': None, 'cycle': 0}

----- Cycle 1 -----
Internal State BEFORE Action:
{'weather': 'rain', 'temperature': 'mild', 'soil_moisture': 'wet', 'pressure': 'off', 'cycle': 0}

Chosen Action:
Sprinkler: off
Irrigation Time: 0 minutes

No irrigation required.

-----------------------------------------

Cycle 2 skipped (mild day sufficient irrigation)




================ DAY 2 ================

Initial Internal State:
{'weather': 'clear', 'temperature': 'mild', 'soil_moisture': 'dry', 'pressure': None, 'cycle': 0}

----- Cycle 1 -----
Internal State BEFORE Action:
{'weather': 'clear', 'temperature': 'mild', 'soil_moisture': 'dry', 'pressure': 'low', 'cycle': 0}

Chosen Action:
Sprinkler: on
Irrigation Time: 20 minutes

Sprinkler running...
Watering for 20 minutes...

Internal State AFTER Irrigation: